# Paddock Allocation

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
import os
import numpy as np
import sys

# Try to import rasterio for GeoTIFF creation
RASTERIO_AVAILABLE = False
try:
    # Try importing rasterio
    import rasterio
    from rasterio.transform import from_bounds
    from rasterio.crs import CRS
    RASTERIO_AVAILABLE = True
    print(f"✓ rasterio imported successfully (version {rasterio.__version__})")
except ImportError as e:
    print(f"⚠ Warning: rasterio import failed")
    print(f"  Error: {e}")
    print(f"  Python: {sys.executable}")
    print(f"  Python version: {sys.version}")
    print("\n  Trying to install rasterio...")
    import subprocess
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "rasterio"], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        # Try importing again after installation
        import importlib
        importlib.invalidate_caches()
        import rasterio
        from rasterio.transform import from_bounds
        from rasterio.crs import CRS
        RASTERIO_AVAILABLE = True
        print(f"  ✓ rasterio installed and imported successfully!")
    except Exception as install_error:
        print(f"  ✗ Installation failed: {install_error}")
        print("  Please install manually with: pip install rasterio")
        print("  Then restart the kernel and run this cell again.")
except Exception as e:
    print(f"⚠ Unexpected error importing rasterio: {e}")
    print(f"  Python: {sys.executable}")

⚠ Warning: rasterio import failed
  Error: No module named 'rasterio'
  Python: c:\Users\ibian\anaconda3\python.exe
  Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]

  Trying to install rasterio...
  ✓ rasterio installed and imported successfully!


In [2]:
# Define input and output paths
input_dir = Path(r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\kml")
kml_file = input_dir / "PROJ0025_GIS_Anameka_Farms.kml"

# Find the specific file "Paddock_Coordinates_Wheat" (CSV or Excel)
csv_file = None
excel_file = None

# Look for the specific file name
target_name = "Paddock_Coordinates_Wheat"
csv_file_path = input_dir / f"{target_name}.csv"
excel_file_path = input_dir / f"{target_name}.xlsx"

if csv_file_path.exists():
    csv_file = csv_file_path
    print(f"Found CSV file: {csv_file.name}")
elif excel_file_path.exists():
    excel_file = excel_file_path
    print(f"Found Excel file: {excel_file.name}")
else:
    # Fallback: search for any file with similar name
    csv_files = list(input_dir.glob(f"*{target_name}*.csv"))
    excel_files = list(input_dir.glob(f"*{target_name}*.xlsx"))
    
    if csv_files:
        csv_file = csv_files[0]
        print(f"Found CSV file: {csv_file.name}")
    elif excel_files:
        excel_file = excel_files[0]
        print(f"Found Excel file: {excel_file.name}")
    else:
        raise FileNotFoundError(f"File '{target_name}' not found in the directory")

# Output path
output_kml = input_dir / "PROJ0025_GIS_Anameka_Farms_with_points.kml"

Found Excel file: Paddock_Coordinates_Wheat.xlsx


In [3]:
# Read the existing KML file
with open(kml_file, 'r', encoding='utf-8') as f:
    kml_content = f.read()

print(f"KML file read successfully: {kml_file.name}")
print(f"File size: {len(kml_content)} characters")

# Parse with ElementTree to understand structure and count placemarks
tree = ET.parse(kml_file)
root = tree.getroot()

# Count existing placemarks
namespace = {'kml': 'http://www.opengis.net/kml/2.2'}
existing_placemarks = root.findall('.//kml:Placemark', namespace)
print(f"Found {len(existing_placemarks)} existing placemarks in the KML file")

KML file read successfully: PROJ0025_GIS_Anameka_Farms.kml
File size: 42138 characters
Found 2 existing placemarks in the KML file


In [4]:
# Read CSV or Excel file - using columns G and H (index 6 and 7)
if csv_file:
    df = pd.read_csv(csv_file)
    print(f"Reading CSV file: {csv_file.name}")
else:
    df = pd.read_excel(excel_file, engine='openpyxl')
    print(f"Reading Excel file: {excel_file.name}")

print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())

# Use columns G and H directly (0-based index: 6 and 7)
# Column G is index 6, Column H is index 7
col_g_index = 6  # Column G
col_h_index = 7  # Column H

# Check if we have enough columns
if len(df.columns) <= col_g_index or len(df.columns) <= col_h_index:
    raise ValueError(f"File does not have enough columns. Need at least columns G and H (indices 6 and 7), but file has {len(df.columns)} columns")

# Get column names for G and H
lat_col = df.columns[col_g_index]  # Column G
lon_col = df.columns[col_h_index]   # Column H

print(f"\nUsing columns G and H:")
print(f"  Column G (index {col_g_index}): {lat_col}")
print(f"  Column H (index {col_h_index}): {lon_col}")

# Extract coordinates and validate
# Handle column name with period if present
df_coords = df[[lat_col, lon_col]].copy()
df_coords.columns = ['latitude', 'longitude']

# Clean up any trailing periods or spaces in coordinate values
df_coords['latitude'] = pd.to_numeric(df_coords['latitude'], errors='coerce')
df_coords['longitude'] = pd.to_numeric(df_coords['longitude'], errors='coerce')

# Remove rows with missing coordinates
df_coords = df_coords.dropna()
print(f"\nValid coordinate pairs: {len(df_coords)}")

# Validate coordinate ranges (basic sanity check)
df_coords = df_coords[
    (df_coords['latitude'] >= -90) & (df_coords['latitude'] <= 90) &
    (df_coords['longitude'] >= -180) & (df_coords['longitude'] <= 180)
]
print(f"Coordinates within valid ranges: {len(df_coords)}")

# Check if there are name/description columns
name_col = None
desc_col = None
for col in df.columns:
    col_lower = str(col).lower().strip()
    if col_lower in ['name', 'id', 'label', 'point_name']:
        name_col = col
    if col_lower in ['description', 'desc', 'note', 'comment']:
        desc_col = col

print(f"\nAdditional columns found:")
if name_col:
    print(f"  Name column: {name_col}")
if desc_col:
    print(f"  Description column: {desc_col}")

Reading Excel file: Paddock_Coordinates_Wheat.xlsx

DataFrame shape: (75, 8)

Column names: ['Unit ID', 'Paddock Name', 'Area (ha)', 'Historical Yield – WHEAT (t/ha)', 'SD Yield – WHEAT (t/ha)', 'Mean Profit – WHEAT ($/ha)', 'Latitude', 'Longitude.']

First few rows:
   Unit ID Paddock Name  Area (ha)  Historical Yield – WHEAT (t/ha)  \
0        1     1080-123     206.61                         2.402039   
1        2      1080-45     203.05                         1.701746   
2        3       1080-6     112.66                         1.853319   
3        4       1080-7      83.19                         1.462928   
4        5        2 Dam      83.19                         1.638931   

   SD Yield – WHEAT (t/ha)  Mean Profit – WHEAT ($/ha)  Latitude  Longitude.  
0                 0.625661                  470.324532  -31.5725    117.4050  
1                 0.534778                  331.608225  -31.5580    117.3900  
2                 0.476262                  294.843775  -31.5580    

In [5]:
# Generate KML Point placemarks for CSV locations
def generate_point_placemarks(df_coords, df_original, name_col=None, desc_col=None):
    """
    Generate KML Point placemarks for each location in the dataframe.
    """
    placemarks = []
    
    for idx, row in df_coords.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        
        # Get name and description if available
        name = f"Point {idx + 1}"
        description = f"Latitude: {lat:.6f}, Longitude: {lon:.6f}"
        
        if name_col and name_col in df_original.columns:
            name_val = df_original.loc[idx, name_col]
            if pd.notna(name_val):
                name = str(name_val)
        
        if desc_col and desc_col in df_original.columns:
            desc_val = df_original.loc[idx, desc_col]
            if pd.notna(desc_val):
                description = str(desc_val)
        
        # Create placemark XML
        placemark = f'''	<Placemark>
		<name>{name}</name>
		<description>{description}</description>
		<styleUrl>#csvPointStyle</styleUrl>
		<Point>
			<coordinates>{lon},{lat},0</coordinates>
		</Point>
	</Placemark>
'''
        placemarks.append(placemark)
    
    return placemarks

# Generate point placemarks
point_placemarks = generate_point_placemarks(df_coords, df, name_col, desc_col)
print(f"Generated {len(point_placemarks)} point placemarks")

Generated 75 point placemarks


In [6]:
# Merge original KML with new point markers
# First, add a style for CSV points (insert after existing styles, before the first Folder or Placemark)
csv_point_style = '''	<Style id="csvPointStyle">
		<IconStyle>
			<scale>1.2</scale>
			<Icon>
				<href>http://maps.google.com/mapfiles/kml/pushpin/red-pushpin.png</href>
			</Icon>
			<hotSpot x="20" y="2" xunits="pixels" yunits="pixels"/>
		</IconStyle>
		<LabelStyle>
			<scale>1.0</scale>
		</LabelStyle>
	</Style>
'''

# Find where to insert the style (after the last Style or StyleMap, before Folder/Placemark)
# Find the last occurrence of </Style> or </StyleMap>
style_insert_pos = kml_content.rfind('</Style>')
if style_insert_pos == -1:
    style_insert_pos = kml_content.rfind('</StyleMap>')

if style_insert_pos != -1:
    # Find the end of the style tag (including closing tag)
    end_pos = kml_content.find('>', style_insert_pos) + 1
    # Insert newline and style after the last style
    kml_content = kml_content[:end_pos] + '\n' + csv_point_style + kml_content[end_pos:]
else:
    # If no style found, insert before first Folder or Placemark
    folder_pos = kml_content.find('<Folder>')
    placemark_pos = kml_content.find('<Placemark>')
    insert_pos = min(folder_pos, placemark_pos) if folder_pos != -1 and placemark_pos != -1 else (folder_pos if folder_pos != -1 else placemark_pos)
    if insert_pos != -1:
        kml_content = kml_content[:insert_pos] + csv_point_style + '\n' + kml_content[insert_pos:]

# Now insert the point placemarks before the closing </Document> tag
# Create a Folder for the CSV points (optional, but keeps things organized)
csv_points_folder = f'''	<Folder>
		<name>CSV Location Points</name>
		<open>1</open>
{''.join(point_placemarks)}	</Folder>
'''

# Find the closing </Document> tag and insert before it
doc_close_pos = kml_content.rfind('</Document>')
if doc_close_pos != -1:
    kml_content = kml_content[:doc_close_pos] + csv_points_folder + '\n' + kml_content[doc_close_pos:]
else:
    raise ValueError("Could not find </Document> closing tag in KML file")

print("KML content merged successfully")

KML content merged successfully


In [7]:
# Write the merged KML file
with open(output_kml, 'w', encoding='utf-8') as f:
    f.write(kml_content)

print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"Input KML file: {kml_file.name}")
if csv_file:
    print(f"Input CSV file: {csv_file.name}")
else:
    print(f"Input Excel file: {excel_file.name}")
print(f"Output KML file: {output_kml.name}")
print(f"\nOriginal placemarks in KML: {len(existing_placemarks)}")
print(f"New point markers added: {len(point_placemarks)}")
print(f"Total placemarks in output: {len(existing_placemarks) + len(point_placemarks)}")
print(f"\nOutput file saved to: {output_kml}")
print(f"\nAdditional formats will be created in the next cell...")
print(f"{'='*60}")


SUMMARY
Input KML file: PROJ0025_GIS_Anameka_Farms.kml
Input Excel file: Paddock_Coordinates_Wheat.xlsx
Output KML file: PROJ0025_GIS_Anameka_Farms_with_points.kml

Original placemarks in KML: 2
New point markers added: 75
Total placemarks in output: 77

Output file saved to: C:\Users\ibian\Desktop\ClimAdapt\Anameka\kml\PROJ0025_GIS_Anameka_Farms_with_points.kml

Additional formats will be created in the next cell...


In [8]:
# Create GeoTIFF file from point coordinates
if RASTERIO_AVAILABLE and len(df_coords) > 0:
    # Calculate bounding box with some padding
    min_lon = df_coords['longitude'].min()
    max_lon = df_coords['longitude'].max()
    min_lat = df_coords['latitude'].min()
    max_lat = df_coords['latitude'].max()
    
    # Add padding (5% of the range)
    lon_range_original = max_lon - min_lon
    lat_range_original = max_lat - min_lat
    padding_lon = max(lon_range_original * 0.05, 0.01)  # At least 0.01 degrees
    padding_lat = max(lat_range_original * 0.05, 0.01)
    
    min_lon -= padding_lon
    max_lon += padding_lon
    min_lat -= padding_lat
    max_lat += padding_lat
    
    # Calculate padded ranges for resolution calculation
    lon_range_padded = max_lon - min_lon
    lat_range_padded = max_lat - min_lat
    
    # Define raster resolution (pixels per degree)
    # Adjust resolution based on original data extent (before padding)
    # Target: at least 500 pixels across the data extent
    resolution = max(lon_range_original / 500, lat_range_original / 500, 0.001)  # At least 500 pixels or 0.001 degree resolution
    
    # Calculate raster dimensions using padded bounds
    width = int((max_lon - min_lon) / resolution)
    height = int((max_lat - min_lat) / resolution)
    
    # Ensure minimum size
    width = max(width, 100)
    height = max(height, 100)
    
    print(f"\nCreating GeoTIFF:")
    print(f"  Bounding box: ({min_lat:.6f}, {min_lon:.6f}) to ({max_lat:.6f}, {max_lon:.6f})")
    print(f"  Resolution: {resolution:.6f} degrees")
    print(f"  Raster size: {width} x {height} pixels")
    
    # Create empty raster array
    raster = np.zeros((height, width), dtype=np.float32)
    
    # Rasterize points (create a density/binary map)
    for idx, row in df_coords.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        
        # Skip if coordinates are NaN
        if pd.isna(lat) or pd.isna(lon):
            continue
        
        # Convert lat/lon to pixel coordinates
        col = int((lon - min_lon) / resolution)
        row_idx = int((max_lat - lat) / resolution)  # Note: lat is inverted in raster coordinates
        
        # Ensure within bounds
        if 0 <= col < width and 0 <= row_idx < height:
            raster[row_idx, col] = 1.0  # Mark point location
    
    # Create transform for georeferencing
    transform = from_bounds(min_lon, min_lat, max_lon, max_lat, width, height)
    
    # Define output GeoTIFF path
    output_geotiff = input_dir / "Paddock_Coordinates_Wheat.tif"
    
    # Write GeoTIFF
    try:
        with rasterio.open(
            output_geotiff,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=1,
            dtype=raster.dtype,
            crs=CRS.from_epsg(4326),  # WGS84
            transform=transform,
            compress='lzw'  # Compression to reduce file size
        ) as dst:
            dst.write(raster, 1)
        
        print(f"  ✓ GeoTIFF saved to: {output_geotiff}")
    except Exception as e:
        print(f"  ✗ Error creating GeoTIFF: {e}")
    
    # Also create JSON format (GeoJSON)
    try:
        import json
        geojson_data = {
            "type": "FeatureCollection",
            "features": []
        }
        
        for idx, row in df_coords.iterrows():
            lat = row['latitude']
            lon = row['longitude']
            
            # Skip if coordinates are NaN
            if pd.isna(lat) or pd.isna(lon):
                continue
            
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [float(lon), float(lat)]
                },
                "properties": {}
            }
            if name_col and name_col in df.columns:
                name_val = df.loc[idx, name_col]
                if pd.notna(name_val):
                    feature["properties"]["name"] = str(name_val)
            geojson_data["features"].append(feature)
        
        output_json = input_dir / "Paddock_Coordinates_Wheat.json"
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(geojson_data, f, indent=2)
        
        print(f"  ✓ GeoJSON saved to: {output_json}")
    except Exception as e:
        print(f"  ✗ Error creating GeoJSON: {e}")
    
else:
    if not RASTERIO_AVAILABLE:
        print("\n⚠ Skipping GeoTIFF creation: rasterio library not available.")
        print("   Please restart the kernel and run all cells again after installing rasterio.")
        print("   Install with: pip install rasterio")
    elif len(df_coords) == 0:
        print("\n⚠ Skipping GeoTIFF creation: No valid coordinates found.")


Creating GeoTIFF:
  Bounding box: (-34.926500, 117.367600) to (34.901500, 117.684400)
  Resolution: 0.126960 degrees
  Raster size: 100 x 550 pixels
  ✓ GeoTIFF saved to: C:\Users\ibian\Desktop\ClimAdapt\Anameka\kml\Paddock_Coordinates_Wheat.tif
  ✓ GeoJSON saved to: C:\Users\ibian\Desktop\ClimAdapt\Anameka\kml\Paddock_Coordinates_Wheat.json
